<a href="https://colab.research.google.com/github/heitorabqg/datascientist/blob/master/PSI5892_proj_1121.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import h5py
import scipy.io as io
import sklearn

import kagglehub

# Download latest version
path = kagglehub.dataset_download("khilian/deepradar")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'deepradar' dataset.
Path to dataset files: /kaggle/input/deepradar


In [ ]:
your_path = '/kaggle/input/deepradar/'
classes = ['LFM','2FSK','4FSK','8FSK', 'Costas','2PSK','4PSK','8PSK','Barker','Huffman','Frank','P1','P2','P3','P4','Px','Zadoff-Chu','T1','T2','T3','T4','NM','Noise']

In [ ]:
with h5py.File(your_path +'X_train.mat', 'r') as f:
    X_train = np.array(f['X_train'], dtype='float32').T
with h5py.File(your_path +'X_val.mat', 'r') as f:
    X_val = np.array(f['X_val'], dtype='float32').T
with h5py.File(your_path +'X_test.mat', 'r') as f:
    X_test = np.array(f['X_test'], dtype='float32').T

In [ ]:
Y_train = io.loadmat(dataset_path + 'Y_train.mat')['Y_train']
Y_val = io.loadmat(dataset_path + 'Y_val.mat')['Y_val']
Y_test = io.loadmat(dataset_path + 'Y_test.mat')['Y_test']
lbl_train = io.loadmat(dataset_path + 'lbl_train.mat')['lbl_train']
lbl_val = io.loadmat(dataset_path + 'lbl_val.mat')['lbl_val']
lbl_test = io.loadmat(dataset_path + 'lbl_test.mat')['lbl_test']

NameError: name 'dataset_path' is not defined

In [ ]:
np.random.seed(1961)
X_train, Y_train,lbl_train = sklearn.utils.shuffle(X_train[:], Y_train[:] ,lbl_train[:],random_state=1961)
X_val, Y_val , lbl_val = sklearn.utils.shuffle(X_val[:], Y_val[:], lbl_val[:],random_state=1961)
X_test, Y_test , lbl_test= sklearn.utils.shuffle(X_test[:], Y_test[:],lbl_test[:] ,random_state=1961)

In [ ]:
#### HA

In [ ]:
# ============================================================
# 1. IMPORTS
# ============================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import numpy as np
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================================================
# 2. LOAD DATA (adapt paths if needed)
# ============================================================

DATA_PATH = '/kaggle/input/deepradar/'   # Kaggle auto-mount path

signals_path = os.path.join(DATA_PATH, "signals.npy")
labels_path  = os.path.join(DATA_PATH, "labels.npy")

print("Loading signals...")
X = np.load(signals_path)
y = np.load(labels_path)

X = X.astype(np.float32)
y = y.astype(np.int64)

signal_length = X.shape[1]
num_classes = len(np.unique(y))

print("Data loaded.")
print("Signal length:", signal_length)
print("Classes:", num_classes)
print("Samples:", len(X))

# ============================================================
# 3. CREATE DATASET + DATALOADER
# ============================================================

# Convert to PyTorch tensors
X_tensor = torch.tensor(X)
y_tensor = torch.tensor(y)

dataset = TensorDataset(X_tensor, y_tensor)

# Split into train/val
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

batch_size = 256

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

print("Train batches:", len(train_loader))
print("Val batches:", len(val_loader))

# ============================================================
# 4. DEFINE MLP MODEL
# ============================================================

class MLPClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MLPClassifier, self).__init__()

        self.fc1 = nn.Linear(input_dim, 512)
        self.bn1 = nn.BatchNorm1d(512)

        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)

        self.fc3 = nn.Linear(256, num_classes)

        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

model = MLPClassifier(signal_length, num_classes).to(device)
print(model)

# ============================================================
# 5. LOSS + OPTIMIZER
# ============================================================

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# ============================================================
# 6. TRAINING FUNCTIONS
# ============================================================

def train_epoch(model, loader, optimizer):
    model.train()
    total_loss = 0

    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)

        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    loss_sum = 0

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss_sum += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

    return loss_sum / len(loader), correct / total

# ============================================================
# 7. TRAIN LOOP
# ============================================================

epochs = 20

for epoch in range(epochs):
    train_loss = train_epoch(model, train_loader, optimizer)
    val_loss, val_acc = evaluate(model, val_loader)

    print(f"Epoch {epoch+1}/{epochs} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | "
          f"Val Acc: {val_acc:.4f}")

# ============================================================
# 8. FINISHED
# ============================================================

print("Training complete.")


Using device: cpu
Loading signals...


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/deepradar/signals.npy'